In [ ]:
%matplotlib inline

# Use a Pydantic grammar.


In [ ]:
from __future__ import annotations

import contextlib

from numpy import array
from pydantic import BaseModel
from pydantic import Field

from gemseo.core.grammars.errors import InvalidDataError
from gemseo.core.grammars.pydantic_grammar import PydanticGrammar
from gemseo.utils.pydantic_ndarray import NDArrayPydantic

## Create the Pydantic model


The Pydantic model is a class deriving from `pydantic.BaseModel`
that describes the names and types of the data to be validated.
Descriptions are defined with docstrings, default values can be defined naturally.
Mind that default values with a mutable object must be defined with the
`default_factory` of a `Field`.
By default,
Pydantic does not handle the typing of NumPy arrays.
To support it,
a special type shall be used, `NDArrayPydantic`.
Like the standard NumPy type for `ndarray`, `NDArray`,
this type can be specialized with the dtype like `NDArrayPydantic[int]`.



In [ ]:
class Model(BaseModel):
    """The description of the model."""

    a_int: int
    """The description of an integer."""

    an_ndarray: NDArrayPydantic
    """The description of an ndarray."""

    an_ndarray_of_int: NDArrayPydantic[int]
    """The description of an ndarray of integers."""

    an_ndarray_with_default: NDArrayPydantic = Field(default_factory=lambda: array([0]))
    """The description of an ndarray with a default value."""

    a_str_with_default: str = "default"
    """The description of a string with a default value."""

## Create the grammar




In [ ]:
grammar = PydanticGrammar("grammar", model=Model)

Show the contents of the grammar.



In [ ]:
grammar

## Validate data against the grammar




Validating missing data will raise an error shows the missing required elements,
here the first 3 elements are missing.



In [ ]:
with contextlib.suppress(InvalidDataError):
    grammar.validate({})

Validating data with bad type will raise an error shows the bad elements,
here the first elements shall be an int and the third one shall be a ndarray of int.



In [ ]:
with contextlib.suppress(InvalidDataError):
    grammar.validate({
        "a_int": 0.0,
        "an_ndarray": array([1]),
        "an_ndarray_of_int": array([1.0]),
    })

Validating compliant data.



In [ ]:
grammar.validate({
    "a_int": 0,
    "an_ndarray": array([1]),
    "an_ndarray_of_int": array([1]),
})

## Grammar defaults

As compared to the other types of grammars, the grammar defaults are be defined
in the Pydantic model and does not require to be manually defined from the grammar.



In [ ]:
grammar.defaults

## Model inheritance

Since Pydantic models are classes, one can easily build grammar via inheritance of the
Pydantic model.
Here we change the type of one element, and we add a new one.



In [ ]:
class Model2(Model):
    """A model that inherits from a parent model."""

    an_ndarray: NDArrayPydantic[float] = Field(default_factory=lambda: array([1.0]))
    """The new description of an_ndarray."""

    a_bool: bool = True
    """The description of a_bool."""


grammar = PydanticGrammar("grammar", model=Model2)

Show the contents of the grammar.



In [ ]:
grammar